In [1]:
import os
import pandas as pd

from project.input.resources import resources_data
from project.model import get_inputs
from project.utils import subplots_attributes, subplots_pie, plot_attribute, plot_attribute2attribute
from project.utils import cumulated_plot,cumulated_plots

In [ ]:
output = 'output/description_stock'
if not os.path.isdir(output):
    os.mkdir(output)

## Loading inputs
Inputs come from the Res-IRF reference scenario.

In [ ]:
inputs = get_inputs(variables=['buildings'])
buildings = inputs['buildings']

## General description

In [ ]:
stock = buildings.simplified_stock(energy_level=True)
stock = stock.groupby(['Occupancy status', 'Income owner', 'Income tenant', 'Housing type', 'Energy', 'Performance']).sum()

In [ ]:
subplots_attributes(stock, dict_order=resources_data['index'], dict_color=resources_data['colors'], percent=False, sharey=True)
subplots_attributes(stock, dict_order=resources_data['index'], dict_color=resources_data['colors'], percent=False, sharey=True, save=os.path.join(output, 'stock_sdes2018_all.png'))

In [ ]:
subplots_pie(stock, dict_order=resources_data['index'], pie=['Housing type', 'Energy', 'Occupancy status'], dict_color=resources_data['colors'], percent=False)


In [ ]:
plot_attribute(stock, attribute='Performance', dict_order=resources_data['index'], percent=False, dict_color=resources_data['colors'], width=0.4,
               save='output/stock_sdes2018_energy_performance.png', figsize=(8.0, 6.0))

In [ ]:
plot_attribute2attribute(stock, 'Performance', 'Energy', dict_order=resources_data['index'], dict_color=resources_data['colors'])
plot_attribute2attribute(stock, 'Performance', 'Energy', dict_order=resources_data['index'], dict_color=resources_data['colors'], percent=False)
plot_attribute2attribute(stock, 'Performance', 'Energy', dict_order=resources_data['index'], dict_color=resources_data['colors'], percent=False, save='output/stock_sdes2018_dpe_energy.png')

In [ ]:
plot_attribute2attribute(stock, 'Performance', 'Energy', dict_order=resources_data['index'], dict_color=resources_data['colors'], percent=True)

### Table

Energy by energy performance certificate

In [ ]:
stock.groupby(['Performance', 'Energy']).sum().unstack('Energy')

Energy by energy performance certificate (%)

In [ ]:
(stock.groupby(['Performance', 'Energy']).sum().unstack('Energy').T / stock.groupby(['Performance', 'Energy']).sum().unstack('Energy').sum(axis=1)).T

## Description of the thermal performance

In [ ]:
temp = dict()
for i in ['Wall', 'Floor', 'Roof', 'Windows']:
    y = pd.Series(buildings.stock.index.get_level_values(i), index=buildings.stock.index, name='{} insulation (W/m2.K)'.format(i)).astype('float')
    x = buildings.stock / 10**6
    temp.update({i: cumulated_plot(x, y, plot=False)})


In [ ]:
cumulated_plots(temp, 'Thermal transmittance U (W/m2.K)', ylim=3)

In [ ]:
stock.groupby(['Housing type', 'Occupancy status', 'Performance']).sum().to_csv(os.path.join(output, 'stock_grouped.csv'))